## Dow Jones Industrial Average (DJIA) index movement prediction using Daily News from Reddit and past Historical Data

### Name: Aristotelis-Angelos Papadopoulos
### USC ID: 3804-2945-23

In [225]:
# Import the required libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import LogisticRegression

from numpy import nan as Nan
# In order to supress some warnings
import warnings
warnings.filterwarnings("ignore")

## 1. Import Data and Preprocessing

In [226]:
# Create a dataframe with the stock data
# This dataframe contains the 'Date', the 'Open', 'High', 'Low', 'Close' and the 'Adjusting Close' 
# prices of the stock as well as the volume of the stock from 2008 up to 2016. 
stock_data = pd.read_csv('DJIA_table.csv', sep = ",")

In order to deal with the Reddit news data, we used some preprocessing techniques borrowed from the NLP literature with which, we are able to extract the 'Subjectivity' and the 'Objectivity' of the news as well as the 'Positive', 'Neutral' or 'Negative' sentiment that the news have for the stock price movement. For more details regarding this preprocessing technique, please check the following GitHub [link](https://github.com/AristotelisPap/Stock-Price-Prediction-Model/tree/master/Sentence_Polarity).

In [227]:
# Create a dataframe with the Reddit News as well as the aforementioned extracted features 
news_data = pd.read_csv('combined_stock_data_proc.csv', parse_dates=[0])
del news_data['Unnamed: 0']

Now, I am going to merge the 'Subjectivity', the 'Objectivity', the 'Positive', the 'Neutral' and the 'Negative' feautures of the news_data dataframe with the stock_data dataframe.

In [228]:
# Create a dataframe by merging the headlines and the stock prices dataframe
news_data_sub = news_data.loc[:,["Date", "Subjectivity", "Objectivity", "Positive", "Neutral", "Negative", "Label"]]
dataset = news_data_sub.merge(stock_data, how='inner', on='Date', left_index=True)

Now, at each day, we are going to take into consideration what were the the 'Open', 'High', 'Low' and 'Close' prices of the stock during the past 4 days! Moreover, we will also take into cobsideration what was the movement of the index during those days (whether it increased or decreased). Note that we are going to throw the first 4 data points of the dataset in order to avoid missing values while taking into consideration the prices of the index for the last 4 days! So, let's modify our dataset accordingly! 

In [229]:
# Prices 1 day ago
prices_1_day_ago = dataset.loc[range(1985,0,-1), ["Open", "High", "Low", "Close", "Label"]]

# Prices 2 days ago
prices_2_days_ago = dataset.loc[range(1986,1,-1), ["Open", "High", "Low", "Close", "Label"]]

# Prices 3 days ago
prices_3_days_ago = dataset.loc[range(1987,2,-1), ["Open", "High", "Low", "Close", "Label"]]

# Prices 4 days ago
prices_4_days_ago = dataset.loc[range(1988,3,-1), ["Open", "High", "Low", "Close", "Label"]]

# Now, we can safely delete the first 4 data points of our dataset
dataset = dataset.drop(dataset.index[0:4])

In [230]:
# Now, in order to concatenate the dataframes, we are going to convert
# them into numpy arrays and then we will cocatenate the arrays!

dataset_array = dataset.values
prices_1_day_ago_array = prices_1_day_ago.values
prices_2_days_ago_array = prices_2_days_ago.values
prices_3_days_ago_array = prices_3_days_ago.values
prices_4_days_ago_array = prices_4_days_ago.values

# Now, let's concatenate the arrays
dataset_array = np.concatenate((dataset_array, prices_1_day_ago_array), axis=1)
dataset_array = np.concatenate((dataset_array, prices_2_days_ago_array), axis=1)
dataset_array = np.concatenate((dataset_array, prices_3_days_ago_array), axis=1)
dataset_array = np.concatenate((dataset_array, prices_4_days_ago_array), axis=1)

# Convert the dataset_array into a dataframe
dataset = pd.DataFrame(dataset_array)

In [231]:
# Name the columns of the dataframe
dataset.columns = ['Date', 'Subjectivity', 'Objectivity', 'Positive', 'Neutral', 'Negative', 'Label',
                   'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close', 'Open 1 day ago',
                   'High 1 day ago', 'Low 1 day ago', 'Close 1 day ago', 'Movement 1 day ago', 
                   'Open 2 days ago', 'High 2 days ago', 'Low 2 days ago', 'Close 2 days ago',
                   'Movement 2 days ago', 'Open 3 days ago', 'High 3 days ago', 'Low 3 days ago',
                   'Close 3 days ago', 'Movement 3 days ago', 'Open 4 days ago', 'High 4 days ago',
                   'Low 4 days ago', 'Close 4 days ago', 'Movement 4 days ago']

In [232]:
# Now let's do some final steps as preprocessing in our dataset!

# First, we are going to remove the features 'High', 'Low', 'Close', 'Volume', 'Adj Close'
# from each data point since these are values that are not known during the time of
# the prediction. However, we are going to assume the feature 'Open' is known during the
# time of the prediction!
del dataset['High']
del dataset['Low']
del dataset['Close']
del dataset['Volume']
del dataset['Adj Close']

# As a last step, we are going to move the Label into the last column of our dataframe!
cols = list(dataset)
cols.append(cols.pop(cols.index('Label')))
dataset = dataset.ix[:, cols]
dataset

,Date,Subjectivity,Objectivity,Positive,Neutral,Negative,Open,Open 1 day ago,High 1 day ago,Low 1 day ago,...,High 3 days ago,Low 3 days ago,Close 3 days ago,Movement 3 days ago,Open 4 days ago,High 4 days ago,Low 4 days ago,Close 4 days ago,Movement 4 days ago,Label
0,2008-08-14,45.4545,54.5455,36.3636,54.5455,9.09091,11532.1,11632.8,11633.8,11453.3,...,11867.1,11675.5,11782.3,1,11432.1,11760,11388,11734.3,0,1
1,2008-08-15,70,30,10,30,60,11611.2,11532.1,11718.3,11450.9,...,11782.3,11601.5,11642.5,0,11729.7,11867.1,11675.5,11782.3,1,1
2,2008-08-18,100,0,0,0,100,11659.7,11611.2,11709.9,11599.7,...,11633.8,11453.3,11533,0,11781.7,11782.3,11601.5,11642.5,0,0
3,2008-08-19,22.2222,77.7778,22.2222,77.7778,0,11478.1,11659.7,11690.4,11434.1,...,11718.3,11450.9,11615.9,1,11632.8,11633.8,11453.3,11533,0,0
4,2008-08-20,70,30,10,30,60,11345.9,11478.1,11478.2,11318.5,...,11709.9,11599.7,11659.9,1,11532.1,11718.3,11450.9,11615.9,1,1
5,2008-08-21,50,50,20,50,30,11415.2,11345.9,11454.2,11290.6,...,11690.4,11434.1,11479.4,0,11611.2,11709.9,11599.7,11659.9,1,1
6,2008-08-22,50,50,0,50,50,11426.8,11415.2,11476.2,11315.6,...,11478.2,11318.5,11348.5,0,11659.7,11690.4,11434.1,11479.4,0,1
7,2008-08-25,55.5556,44.4444,22.2222,44.4444,33.3333,11626.2,11426.8,11632.1,11426.8,...,11454.2,11290.6,11417.4,1,11478.1,11478.2,11318.5,11348.5,0,0
8,2008-08-26,66.6667,33.3333,0,33.3333,66.6667,11383.6,11626.2,11626.3,11362.6,...,11476.2,11315.6,11430.2,1,11345.9,11454.2,11290.6,11417.4,1,1
9,2008-08-27,30.7692,69.2308,30.7692,69.2308,0,11412.5,11383.6,11436.2,11340.4,...,11632.1,11426.8,11628.1,1,11415.2,11476.2,11315.6,11430.2,1,1


In [233]:
# Now, let us convert the type of our data from object to float such that we will be able to directly use the 
# statistics of each feature later!

dataset['Subjectivity'] = dataset['Subjectivity'].astype(float)
dataset['Objectivity'] = dataset['Objectivity'].astype(float)
dataset['Positive'] = dataset['Positive'].astype(float)
dataset['Neutral'] = dataset['Neutral'].astype(float)
dataset['Negative'] = dataset['Negative'].astype(float)
dataset['Open'] = dataset['Open'].astype(float)
dataset['Open 1 day ago'] = dataset['Open 1 day ago'].astype(float)
dataset['High 1 day ago'] = dataset['High 1 day ago'].astype(float)
dataset['Low 1 day ago'] = dataset['Low 1 day ago'].astype(float)
dataset['Close 1 day ago'] = dataset['Close 1 day ago'].astype(float)
dataset['Movement 1 day ago'] = dataset['Movement 1 day ago'].astype(float)
dataset['Open 2 days ago'] = dataset['Open 2 days ago'].astype(float)
dataset['High 2 days ago'] = dataset['High 2 days ago'].astype(float)
dataset['Low 2 days ago'] = dataset['Low 2 days ago'].astype(float)
dataset['Close 2 days ago'] = dataset['Close 2 days ago'].astype(float)
dataset['Movement 2 days ago'] = dataset['Movement 2 days ago'].astype(float)
dataset['Open 3 days ago'] = dataset['Open 3 days ago'].astype(float)
dataset['High 3 days ago'] = dataset['High 3 days ago'].astype(float)
dataset['Low 3 days ago'] = dataset['Low 3 days ago'].astype(float)
dataset['Close 3 days ago'] = dataset['Close 3 days ago'].astype(float)
dataset['Movement 3 days ago'] = dataset['Movement 3 days ago'].astype(float)
dataset['Open 4 days ago'] = dataset['Open 4 days ago'].astype(float)
dataset['High 4 days ago'] = dataset['High 4 days ago'].astype(float)
dataset['Low 4 days ago'] = dataset['Low 4 days ago'].astype(float)
dataset['Close 4 days ago'] = dataset['Close 4 days ago'].astype(float)
dataset['Movement 4 days ago'] = dataset['Movement 4 days ago'].astype(float)
dataset['Label'] = dataset['Label'].astype(int)


# Now, let's print the description of our dataset, i.e. the statistics of each feature in our dataset!
print(dataset.describe())

       Subjectivity  Objectivity     Positive      Neutral     Negative  \
count   1982.000000  1982.000000  1982.000000  1982.000000  1982.000000   
mean      56.699306    43.300694    19.959309    43.300694    36.739996   
std       21.563572    21.563572    16.382322    21.563572    21.124198   
min        0.000000     0.000000     0.000000     0.000000     0.000000   
25%       40.000000    28.571429     8.333333    28.571429    21.739130   
50%       54.545455    45.454545    18.181818    45.454545    33.333333   
75%       71.428571    60.000000    28.571429    60.000000    50.000000   
max      100.000000   100.000000   100.000000   100.000000   100.000000   

               Open  Open 1 day ago  High 1 day ago  Low 1 day ago  \
count   1985.000000     1985.000000     1985.000000    1985.000000   
mean   13462.773577    13459.604091    13541.682725   13373.390028   
std     3145.385183     3144.056804     3137.105163    3151.148286   
min     6547.009766     6547.009766     6709

## 2. Train, Validation and Test set

As it is also recommended by the Kaggle website ([link](https://www.kaggle.com/aaron7sun/stocknews/home)), we are going to use data from 2008-08-08 to 2014-12-31 as Training Set and Test Set is then the following two years data (from 2015-01-02 to 2016-07-01). Since our data consist of time-series, we are not able to use a Cross-Validation approach. Therefore, we are going to extract some data from the Training set in order to use them as a Validation Set for hyperparameter tuning of our machine learning algorithms. So, finally, we are going to have:

#### Training Set: 2008 - 2013
#### Validation Set: 2014
#### Test Set: 2015 - 2016

In [234]:
training_set = dataset.iloc[0:1355]
validation_set = dataset.iloc[1355:1607]
test_set = dataset.iloc[1607:]

## 3. Machine Learning models

### 3.1. Logistic Regression with L2 regularization

The methodology that I am going to apply will be as follows: I am going to use the validation set in order to pick the best value of the parameter lamda of the regularization. However, I plan to make my algorithm to update its weights every certain number of days such that it can be adapted to new upcoming data. How often the model is going to update its weights, is going to be determined by the validation set!  

In [441]:
# Let's separate the input x from the output y of the training, validation and test sets!
training_x = training_set.loc[:,'Subjectivity':'Movement 4 days ago']
training_y = training_set.loc[:,'Label']
validation_x = validation_set.loc[:,'Subjectivity':'Movement 4 days ago']
validation_y = validation_set.loc[:,'Label']
test_x = test_set.loc[:,'Subjectivity':'Movement 4 days ago']
test_y = test_set.loc[:,'Label']

# Now, let me fill the missing values of the features using their mean!
# Change the NaN values to the mean value of that column
nan_list = ['Subjectivity', 'Objectivity', 'Positive', 'Negative', 'Neutral']
for col in nan_list:
    training_x[col] = training_x[col].fillna(training_x[col].mean())
    validation_x[col] = validation_x[col].fillna(validation_x[col].mean())
    test_x[col] = test_x[col].fillna(test_x[col].mean())

In [442]:
# Before proceeding to the training of the algorithm, I will first define a function which is 
# going to be used for standardizing the data!

def Preprocessing(train_set, test_set, standardization_method):
    """
    Implements Preprocessing of the data using the requested standardization method.
    
    Arguments:
    train_set -- training data, shape (number of training examples, number of features)
    test_set -- test data, shape (number of test examples, number of features)
    standardization_method -- Can be 1 out of the 3 aforementioned methods
    
    Returns:
    train_set_std -- standardized training data
    test_set_std -- standardized test data
    """
    
    train_set_std = np.zeros((train_set.shape[0], train_set.shape[1]))
    test_set_std = np.zeros((test_set.shape[0], test_set.shape[1]))
    
    if standardization_method == 'standard':
        # Now, I have to standardize my training set such that every column 
        # has zero mean and unit variance!
        mean = np.mean(train_set, axis=0)
        standard_deviation = np.std(train_set, axis=0)
        for i in range (0, train_set.shape[0]):
            for j in range(0, train_set.shape[1]):
                train_set_std[i,j] = (train_set[i,j] - mean[j]) / standard_deviation[j] 
        # Now, I also have to do the standardization to the test set as well,
        # by using the mean and the standard deviation calculated above.
        for i in range (0, test_set.shape[0]):
            for j in range(0, test_set.shape[1]):
                    test_set_std[i,j] = (test_set[i,j] - mean[j]) / standard_deviation[j]
                    
    elif standardization_method == 'log':
        train_set_std = np.log(train_set + 0.1)
        test_set_std = np.log(test_set + 0.1)
        
    elif standardization_method == 'binarize':
        threshold, upper, lower = 0, 1, 0
        train_set_std = np.where(train_set > threshold, upper, lower)
        test_set_std = np.where(test_set > threshold, upper, lower)
        
                    
    return train_set_std, test_set_std

In [456]:
# As it was mentioned before, we are going to use the validation set in order to determine 
# both the best regularization parameter as well as how often the algorithm should
# update its weights according to the new upcoming data! 

# We are going to use 40 different values for C!
C_range = np.logspace(-4, 4, num=40) # C ranges from 10^-4 to 10^4 with a log increment

# The possible update of the weights could happen after 5, 10, 15 or 20 days!
upd_days = [5,10,15,20]

# In this list, I will save the different Validation errors that I am going to compare.
ave_Validation_errors = [] 

for C1 in C_range:
    for days in upd_days:
        training_x = training_set.loc[:,'Subjectivity':'Movement 4 days ago']
        training_y = training_set.loc[:,'Label']
        validation_x = validation_set.loc[:,'Subjectivity':'Movement 4 days ago']
        validation_y = validation_set.loc[:,'Label']
        validation_y = validation_y.values # In order to convert it into a numpy array
        
        # Now, let me fill the missing values of the features using their mean!
        # Change the NaN values to the mean value of that column
        nan_list = ['Subjectivity', 'Objectivity', 'Positive', 'Negative', 'Neutral']
        for col in nan_list:
            training_x[col] = training_x[col].fillna(training_x[col].mean())
            validation_x[col] = validation_x[col].fillna(validation_x[col].mean())
        
        # Standardize the data to have zero mean and unit variance
        training_x, validation_x = Preprocessing(training_x.values, validation_x.values, 'standard')
        
        # Now, I will fit my Logistic Regression model in the Training set!
        model = LogisticRegression(penalty='l2', C=C1)
        model.fit(training_x, training_y)
        
        # Now, in the validation setting, I will add data from the validation set
        # into the training set every certain number of days in order to find
        # how often the model should update its weights!
        
        # First, I will create an empty list where I am going to add the prediction errors
        # after each certain number of days!
        indiv_errors = []
        
        for i in range(0, len(validation_x)):
            if (i+1) % days == 0:
                # Evaluate the performance on the datastream of the validation set
                # and save it to the list!
                datastream_x = validation_x[0:days,:]
                datastream_y = validation_y[0:days]
                indiv_errors.append(1 - model.score(datastream_x, datastream_y))
                # Add the datastream to the training set
                training_x = np.append(training_x, validation_x[0:days,:], axis=0)
                training_y = np.append(training_y, validation_y[0:days], axis=0)
                # Delete the datastream from the validation set
                for i in range(0, days):
                    validation_x = np.delete(validation_x,(0),axis=0)
                    validation_y = np.delete(validation_y,(0),axis=0)
                # Retrain the model on the updated training set
                model.fit(training_x, training_y)
                
        # Calculate the average error of the indiv_errors list and add it
        # to the ave_Validation_errors list!
        ave_Validation_errors.append(sum(indiv_errors) / len(indiv_errors))
        
# Convert the ave_Validation_errors list into a numpy array
Array_valid = np.asarray(ave_Validation_errors)
Array_valid = Array_valid.reshape(40,4) # Indexes are (C, upd_days)
# Take the indexes of the minimum element of the Array_valid
ind_arr_val = np.unravel_index(np.argmin(Array_valid, axis=None), Array_valid.shape)

print("The minimum Validation Error happens for : C =", C_range[ind_arr_val[0]])
print("The algorithm should update its weights every", upd_days[ind_arr_val[1]], "days")
print("The Validation Error is: ", min(ave_Validation_errors))

The minimum Validation Error happens for : C = 2424.462017082326
The algorithm should update its weights every 5 days
The Validation Error is:  0.3719999999999997


In [455]:
# Now, we know what should be the regularization parameter C as well as
# how often the algorithm should update its weights. So, we can evaluate
# the performance of our algorithm!

# Standardize the data to have zero mean and unit variance
#training_x, validation_x = Preprocessing(training_x, validation_x, 'standard')

# Now, I will refit my Logistic Regression model using the optimum C calculated by CV.
model = LogisticRegression(penalty='l2', C= C_range[ind_arr_val[0]])
model.fit(training_x, training_y)

print("The error on the full given training set is: ", 1 - model.score(training_x, training_y))
print("The error on the full given test set is: ", 1 - model.score(test_x, test_y))

The error on the full given training set is:  0.3974921630094044
The error on the full given test set is:  0.3571428571428571
